In [ ]:
import numpy as np 
import torch

def custom_conv_transpose(input_tensor, input_channels, output_channels, kernel_size, 
                          stride=1, padding=0, output_padding=0, dilation=1, 
                          bias=True, padding_mode='zeros'):
    
    bias_values = torch.rand(output_channels) if bias else torch.zeros(output_channels)

    if padding_mode != 'zeros':
        raise ValueError('Only "zeros" padding mode supported')

    weights = torch.rand(input_channels, output_channels, kernel_size, kernel_size) if isinstance(kernel_size, int) else torch.rand(input_channels, output_channels, *kernel_size)

    result_tensors = []

    for out_channel in range(output_channels):
        feature_map = torch.zeros((input_tensor.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,  (input_tensor.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1)
        
        for in_channel in range(input_channels):
            for i in range(0, input_tensor.shape[1]):
                for j in range(0, input_tensor.shape[2]):
                
                    input_value = input_tensor[in_channel][i][j] 
                    product = input_value * weights[in_channel][out_channel]
                
                    zeros_tensor = torch.zeros((weights.shape[2] - 1) * dilation + 1, (weights.shape[3] - 1) * dilation + 1)
                
                    for m in range(0, zeros_tensor.shape[0], dilation):
                        for n in range(0, zeros_tensor.shape[1], dilation):
                            zeros_tensor[m][n] = product[m // dilation][n // dilation]
                
                    summation = np.add(zeros_tensor, 
                                        feature_map[i * stride : i * stride + (weights.shape[2] - 1) * dilation + 1, j * stride : j * stride + (weights.shape[3] - 1) * dilation + 1])
                    feature_map[i * stride : i * stride + (weights.shape[2] - 1) * dilation + 1,  j * stride : j * stride + (weights.shape[3] - 1) * dilation + 1] = summation
                    
        result_tensors.append(np.add(feature_map, np.full(feature_map.shape, bias_values[out_channel])))

    for tensor in range(len(result_tensors)):
        if output_padding > 0:
            pad = torch.nn.ConstantPad2d((0, output_padding, 0, output_padding), 0)
            result_tensors[tensor] = pad(result_tensors[tensor])
            
        result_tensors[tensor] = result_tensors[tensor][padding:result_tensors[tensor].shape[0] - padding, padding:result_tensors[tensor].shape[1] - padding]

    return result_tensors, weights, torch.tensor(bias_values)